In [1]:
import numpy as np
import pandas as pd

In [2]:
from keras.layers.core import Activation, Dense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Dropout
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import nltk  
import collections  
import numpy as np

Using TensorFlow backend.


In [3]:
df1 = pd.read_csv('processed_million2.csv')

In [4]:
df2 = pd.read_csv('test_processed.csv')

In [5]:
text = list(df1.text) + list(df2.text)

In [6]:
maxlen = 0 
word_freqs = collections.Counter()  
for sentence in text:
        words = nltk.word_tokenize(str(sentence))
        if len(words) > maxlen:
            maxlen = len(words)
        for word in words:
            word_freqs[word] += 1

In [7]:
max_features = 617000
max_length = 660

In [8]:
word_no = {x[0]: i+2 for i, x in enumerate(word_freqs.most_common(max_features))}
word_no['pading'] = 0
word_no['not_in_voc'] = 1

In [10]:
X = []
for sentence in text:
    seq = []
    words = nltk.word_tokenize(str(sentence))
    for word in words:
            if word in word_no:
                seq.append(word_no[word])
            else:
                seq.append(word_no['not_in_voc'])
    X.append(seq)

In [11]:
X = sequence.pad_sequences(X, maxlen=max_length)

In [12]:
train = X[:999989]
test = X[999989:]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(train, df1['stars'], test_size=0.3, random_state=123)

In [ ]:
EMBEDDING_SIZE = 256
HIDDEN_LAYER_SIZE = 64

model = Sequential() 
model.add(Embedding(max_features+2, EMBEDDING_SIZE,input_length=max_length))
model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.3, recurrent_dropout=0.3))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss="mean_squared_error", optimizer="adam",metrics=["mean_squared_error"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
BATCH_SIZE = 1024
NUM_EPOCHS = 3 
model.fit(train, df1['stars'], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
166912/999989 [====>.........................] - ETA: 41:09:34 - loss: 2.7681 - mean_squared_error: 2.7681

In [ ]:
pre = model.predict(test)